In [ ]:
#| default_exp utils_seo_tests

In [ ]:
from fh_saas.utils_seo import generate_head_tags, generate_sitemap_xml, generate_rss_xml
from datetime import datetime
import pytest

## Meta Tags Tests

In [ ]:
#| hide

def test_generate_head_tags_basic():
    """Test basic meta tag generation"""
    tags = generate_head_tags(
        title='My Post',
        description='This is a post',
        url='https://example.com/blog/my-post'
    )
    
    tags_dict = {tag: attrs for tag, attrs in tags}
    
    # Check title
    assert any(attrs.get('content') == 'My Post' for _, attrs in tags)
    
    # Check description
    assert any(
        attrs.get('name') == 'description' and attrs.get('content') == 'This is a post'
        for _, attrs in tags
    )
    
    # Check canonical URL
    assert any(
        attrs.get('rel') == 'canonical' and attrs.get('href') == 'https://example.com/blog/my-post'
        for _, attrs in tags
    )

In [ ]:
#| hide

def test_generate_head_tags_opengraph():
    """Test OpenGraph meta tags"""
    tags = generate_head_tags(
        title='My Post',
        description='Description',
        url='https://example.com/post',
        image_url='https://example.com/image.jpg'
    )
    
    # Check og:title
    assert any(
        attrs.get('property') == 'og:title' and attrs.get('content') == 'My Post'
        for _, attrs in tags
    )
    
    # Check og:type
    assert any(
        attrs.get('property') == 'og:type' and attrs.get('content') == 'article'
        for _, attrs in tags
    )
    
    # Check og:image
    assert any(
        attrs.get('property') == 'og:image' and attrs.get('content') == 'https://example.com/image.jpg'
        for _, attrs in tags
    )

In [ ]:
#| hide

def test_generate_head_tags_twitter():
    """Test Twitter Card meta tags"""
    tags = generate_head_tags(
        title='My Post',
        description='Description',
        url='https://example.com/post',
        image_url='https://example.com/image.jpg'
    )
    
    # Check twitter:card
    assert any(
        attrs.get('name') == 'twitter:card' and attrs.get('content') == 'summary_large_image'
        for _, attrs in tags
    )
    
    # Check twitter:image
    assert any(
        attrs.get('name') == 'twitter:image' and attrs.get('content') == 'https://example.com/image.jpg'
        for _, attrs in tags
    )

In [ ]:
#| hide

def test_generate_head_tags_article_metadata():
    """Test article metadata (published, modified, author)"""
    pub_date = datetime(2024, 1, 15, 10, 0)
    mod_date = datetime(2024, 1, 20, 15, 30)
    
    tags = generate_head_tags(
        title='My Post',
        description='Description',
        url='https://example.com/post',
        article_published=pub_date,
        article_modified=mod_date,
        author='Alice'
    )
    
    # Check article:published_time
    assert any(
        attrs.get('property') == 'article:published_time' and '2024-01-15' in attrs.get('content', '')
        for _, attrs in tags
    )
    
    # Check article:modified_time
    assert any(
        attrs.get('property') == 'article:modified_time' and '2024-01-20' in attrs.get('content', '')
        for _, attrs in tags
    )
    
    # Check article:author
    assert any(
        attrs.get('property') == 'article:author' and attrs.get('content') == 'Alice'
        for _, attrs in tags
    )

## Sitemap Tests

In [ ]:
#| hide

def test_generate_sitemap_xml_basic():
    """Test basic sitemap generation"""
    posts = [
        {'slug': 'post1', 'title': 'Post 1', 'date': datetime(2024, 1, 15)},
        {'slug': 'post2', 'title': 'Post 2', 'date': datetime(2024, 1, 20)}
    ]
    
    xml = generate_sitemap_xml(
        posts=posts,
        base_url='https://example.com',
        blog_path='/blog'
    )
    
    # Check XML declaration
    assert '<?xml version="1.0"' in xml
    
    # Check urlset namespace
    assert 'xmlns="http://www.sitemaps.org/schemas/sitemap/0.9"' in xml
    
    # Check blog index
    assert '<loc>https://example.com/blog</loc>' in xml
    
    # Check posts
    assert '<loc>https://example.com/blog/post1</loc>' in xml
    assert '<loc>https://example.com/blog/post2</loc>' in xml

In [ ]:
#| hide

def test_generate_sitemap_xml_lastmod():
    """Test sitemap includes lastmod dates"""
    posts = [
        {'slug': 'post1', 'title': 'Post 1', 'date': datetime(2024, 1, 15)}
    ]
    
    xml = generate_sitemap_xml(
        posts=posts,
        base_url='https://example.com'
    )
    
    assert '<lastmod>2024-01-15</lastmod>' in xml

In [ ]:
#| hide

def test_generate_sitemap_xml_escaping():
    """Test sitemap escapes special characters in URLs"""
    posts = [
        {'slug': 'post&test', 'title': 'Post', 'date': None}
    ]
    
    xml = generate_sitemap_xml(
        posts=posts,
        base_url='https://example.com'
    )
    
    # Should escape & as &amp;
    assert '&amp;' in xml or 'post&test' in xml

## RSS Feed Tests

In [ ]:
#| hide

def test_generate_rss_xml_basic():
    """Test basic RSS feed generation"""
    posts = [
        {
            'slug': 'post1',
            'title': 'First Post',
            'description': 'First post description',
            'date': datetime(2024, 1, 15),
            'author': 'Alice'
        },
        {
            'slug': 'post2',
            'title': 'Second Post',
            'description': 'Second post description',
            'date': datetime(2024, 1, 20)
        }
    ]
    
    xml = generate_rss_xml(
        posts=posts,
        blog_title='My Blog',
        blog_description='Tech Blog',
        base_url='https://example.com',
        blog_path='/blog'
    )
    
    # Check RSS declaration
    assert '<?xml version="1.0"' in xml
    assert '<rss version="2.0"' in xml
    
    # Check channel metadata
    assert '<title>My Blog</title>' in xml
    assert '<description>Tech Blog</description>' in xml
    
    # Check items
    assert '<title>First Post</title>' in xml
    assert '<title>Second Post</title>' in xml
    assert '<link>https://example.com/blog/post1</link>' in xml

In [ ]:
#| hide

def test_generate_rss_xml_pubdate():
    """Test RSS includes pubDate in RFC 2822 format"""
    posts = [
        {
            'slug': 'post1',
            'title': 'Post',
            'date': datetime(2024, 1, 15, 10, 0)
        }
    ]
    
    xml = generate_rss_xml(
        posts=posts,
        blog_title='Blog',
        blog_description='Desc',
        base_url='https://example.com'
    )
    
    # Should contain pubDate tag
    assert '<pubDate>' in xml
    # RFC 2822 format includes day of week
    assert 'Mon' in xml or 'Tue' in xml or 'Wed' in xml or 'Thu' in xml or 'Fri' in xml

In [ ]:
#| hide

def test_generate_rss_xml_escaping():
    """Test RSS escapes special characters"""
    posts = [
        {
            'slug': 'post1',
            'title': 'Post & Test',
            'description': 'Desc with <tags>'
        }
    ]
    
    xml = generate_rss_xml(
        posts=posts,
        blog_title='Blog',
        blog_description='Desc',
        base_url='https://example.com'
    )
    
    # Should escape special characters
    assert '&amp;' in xml
    assert '&lt;' in xml or '&gt;' in xml

In [ ]:
#| hide

def test_generate_rss_xml_atom_link():
    """Test RSS includes atom:link for self-reference"""
    posts = []
    
    xml = generate_rss_xml(
        posts=posts,
        blog_title='Blog',
        blog_description='Desc',
        base_url='https://example.com'
    )
    
    # Should include atom namespace and self link
    assert 'xmlns:atom' in xml
    assert 'atom:link' in xml
    assert 'rss.xml' in xml